In [34]:
import pandas as pd

otu_df = pd.read_csv('dataframes/OTU.txt', sep='\t')
hiera_blast = pd.read_csv('dataframes/hiera_BLAST.txt', sep='\t')
meta_map = pd.read_csv('dataframes/MCOSIS_map.txt', sep='\t')
meta = pd.read_csv('dataframes/antibiotics kazakh finished.csv', sep='\t')


In [35]:
meta.head()

,ID,R2,sample,day of research,type of sample 1=stool; 2=swab,antibiotic,intervention,date of birth,sex,DS,...,RANTES (74),TNFα 75 (75),TNFβ 76 (76),VEGF 78 (78),Ig A,IgG1,IgG2,IgG3,IgG4,IgM
0,01-14c_S52_L001_R1_001.fastq.gz,01-14c_S52_L001_R2_001.fastq.gz,1,40,2,Ceftriaxone,yes,23.10.2012,male,bilateral bronchopneumonia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-15c_S56_L001_R1_001.fastq.gz,01-15c_S56_L001_R2_001.fastq.gz,1,45,2,Ceftriaxone,yes,23.10.2012,male,bilateral bronchopneumonia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-16c_S58_L001_R1_001.fastq.gz,01-16c_S58_L001_R2_001.fastq.gz,1,50,2,Ceftriaxone,yes,23.10.2012,male,bilateral bronchopneumonia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-17c_S80_L001_R1_001.fastq.gz,01-17c_S80_L001_R2_001.fastq.gz,1,55,2,Ceftriaxone,yes,23.10.2012,male,bilateral bronchopneumonia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-18c_S45_L001_R1_001.fastq.gz,01-18c_S45_L001_R2_001.fastq.gz,1,60,2,Ceftriaxone,yes,23.10.2012,male,bilateral bronchopneumonia,...,"0,16","2,20","1,44","0,38",8650,73,"474,00","100,00","564,00","109,00"


In [36]:
meta = meta[['ID', 'sample', 'day of research', 'intervention']]
meta_map_expanded = meta_map.set_index('#SampleID').fastqFile.str.split(',', expand=True).stack().reset_index(name='fastqFile').drop('level_1', axis=1)

# Соединение таблиц
merged = pd.merge(meta, meta_map_expanded, left_on='ID', right_on='fastqFile')

# Замена старых значений 'ID' на новые из '#SampleID'
meta['ID'] = merged['#SampleID']

# Показываем результат
meta.head()

,ID,sample,day of research,intervention
0,SMPL0,1,40,yes
1,SMPL1,1,45,yes
2,SMPL2,1,50,yes
3,SMPL3,1,55,yes
4,SMPL4,1,60,yes


In [39]:
# Транспонирование otu_df
otu_df_transposed = otu_df.set_index('OTU').T
otu_df_transposed = otu_df_transposed.reset_index().rename(columns={'index': 'SampleID'})

# Переименование столбцов в meta для соответствия с otu_df_transposed
meta = meta.rename(columns={'ID': 'SampleID', 'intervention': 'GROUP'})

# Объединение таблиц
merged_df = pd.merge(meta, otu_df_transposed, on='SampleID')

# Переименование столбцов OTU
for col in merged_df.columns:
    if 'OTU' in col:
        merged_df = merged_df.rename(columns={col: col.replace('OTU_', 'OTU')})

# Вывод результата
merged_df = merged_df.drop(columns=['sample', 'day of research'])
merged_df.to_csv('dataframes/output/otu/otu_starting.csv', sep='\t')
merged_df.head()

,SampleID,GROUP,OTU1,OTU2,OTU3,OTU4,OTU5,OTU6,OTU7,OTU8,...,OTU4190,OTU4191,OTU4192,OTU4193,OTU4194,OTU4195,OTU4196,OTU4197,OTU4198,OTU4199
0,SMPL0,yes,12,0,26,0,14,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,SMPL1,yes,416,60,672,1,1641,59,0,33,...,0,0,0,0,0,0,0,0,0,0
2,SMPL2,yes,101,8,7,0,567,1,0,11,...,0,0,0,0,0,0,0,0,0,0
3,SMPL3,yes,3161,1505,11,0,3890,1,3,64,...,0,0,0,0,0,0,0,0,0,0
4,SMPL4,yes,15,3,2,8,123,2,1,2,...,0,0,0,0,0,0,0,0,0,0


In [40]:
group_counts = merged_df['GROUP'].value_counts()
group_counts

GROUP
no     60
yes    52
Name: count, dtype: int64

In [41]:
hiera_blast.head()

,OTU,Domain,Phylum,Class,Order,Family,Genus,Species
0,OTU_4155,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,?
1,OTU_4149,Bacteria,Firmicutes,Clostridia,Clostridiales,Ruminococcaceae,Ruminococcus 1,?
2,OTU_4093,Bacteria,Firmicutes,Clostridia,Clostridiales,Family XI,Parvimonas,Parvimonas micra
3,OTU_4078,Bacteria,Firmicutes,Clostridia,Clostridiales,?,?,?
4,OTU_4073,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,?,?
